# 필요한 parameter
##### 상품코드 | 기본분류(1차+2차+3차, 이벤트상품 제외) | 분류명 | 상품유형 | 상품명 | 제조사 | 판매가격 | 기본배송비 | 조회수 | 판매수량 | 업체ID 
#### 이용자가 관심있는/선호하는 상품유형, 판매가격, 기본배송비 데이터를 수집하여,
#### 판매가격, 기본배송비 필터링 (설정한 판매가격, 기본배송비 범위 내여야 함)
#### 상품유형, 제조사, 업체ID가 유사한 상품을 추천하기

In [ ]:
import pandas as pd

In [ ]:
contents = pd.read_excel("콘텐츠 테이블.xlsx")
contents.head()

,상품코드,기본분류,분류명,상품유형,상품명,제조사,판매가격,기본배송비,조회수,판매수량,업체ID
0,157092753,201030,곶감/반건시/감말랭이,NaN,[상주전통곶감] 전통곶감 건시 4kg(65~70과),상주전통곶감,89000,0,0,0,d3f908bbea5d2d7cac866508dff9b2ffdf0088ab55d178...
1,1578008979,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시3호 48구,구릿뜰농원,55000,4000,130,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...
2,1578031964,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시4호 36구,구릿뜰농원,45000,4000,223,1,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...
3,1578034429,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시 낱개 포장 30구,구릿뜰농원,40000,4000,141,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...
4,1578034546,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시 낱개 포장1호,구릿뜰농원,30000,4000,136,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...


# weighted rating을 위한 '조회수' '판매수량' 칼럼 EDA

In [ ]:
contents.describe()

,상품코드,판매가격,기본배송비,조회수,판매수량
count,3.129000e+03,3.129000e+03,3129.000000,3129.000000,3129.000000
mean,1.595076e+09,1.232850e+05,1892.553531,428.522851,22.961649
std,2.695284e+07,1.885154e+06,1686.428119,601.546638,162.884518
min,1.570928e+08,0.000000e+00,0.000000,0.000000,0.000000
25%,1.589525e+09,1.500000e+04,0.000000,145.000000,0.000000
50%,1.597899e+09,2.500000e+04,3000.000000,297.000000,2.000000
75%,1.601017e+09,4.200000e+04,3000.000000,506.000000,12.000000
max,1.609378e+09,7.620000e+07,25000.000000,8573.000000,5689.000000


In [ ]:
contents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품코드    3129 non-null   int64 
 1   기본분류    3129 non-null   object
 2   분류명     3129 non-null   object
 3   상품유형    2785 non-null   object
 4   상품명     3129 non-null   object
 5   제조사     3065 non-null   object
 6   판매가격    3129 non-null   int64 
 7   기본배송비   3129 non-null   int64 
 8   조회수     3129 non-null   int64 
 9   판매수량    3129 non-null   int64 
 10  업체ID    3129 non-null   object
dtypes: int64(5), object(6)
memory usage: 269.0+ KB


In [ ]:
print(contents.isnull().sum())

상품코드       0
기본분류       0
분류명        0
상품유형     344
상품명        0
제조사       64
판매가격       0
기본배송비      0
조회수        0
판매수량       0
업체ID       0
dtype: int64


In [ ]:
contents["상품코드"] = contents["상품코드"].astype("object")
contents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품코드    3129 non-null   object
 1   기본분류    3129 non-null   object
 2   분류명     3129 non-null   object
 3   상품유형    2785 non-null   object
 4   상품명     3129 non-null   object
 5   제조사     3065 non-null   object
 6   판매가격    3129 non-null   int64 
 7   기본배송비   3129 non-null   int64 
 8   조회수     3129 non-null   int64 
 9   판매수량    3129 non-null   int64 
 10  업체ID    3129 non-null   object
dtypes: int64(4), object(7)
memory usage: 269.0+ KB


# weighted rating
### 상품 선호/평점 측도: 판매수량
### 조회수가 선호/평점 측도라는 근거: 조회수
#### 판매수량과 조회수 heatmap 상관계수 EDA 결과 제시
#### r: 개별 영화 평점 -> 개별 상품 판매수량
#### v: 개별 영화 평점 투표수 -> 개별 상품 조회수
#### m: 500위에 들어야 하는 최소 투표 수 -> 500위에 들어야 하는 최소 조회수
#### c:  전체 영화 평균 평점 -> 전체 상품 평균 판매수량

In [ ]:
# 전체 상품 중 약 300위(약 상위10%)에 들려면 v는 상위 몇%여야 할까?
m = contents["조회수"].quantile(0.9)
data = contents.loc[contents["조회수"] >= m]
data.describe
# v는 90%일때 m은 314개

<bound method NDFrame.describe of             상품코드  ...                                               업체ID
37    1587101982  ...  64f45d945a1ff30d8c1eeed085bdf30c551b26bb0de8b9...
40    1589530054  ...  13fd756bafc34715f8c88ca1e09af5f55118684190cc24...
41    1589530697  ...  13fd756bafc34715f8c88ca1e09af5f55118684190cc24...
115   1607064822  ...  f404187bc45f73ce1a32c7d3cdda558aca9aa773a2000e...
157   1584970099  ...  3e736b202d0723b960b471cd4aad470d38a4665ead176e...
...          ...  ...                                                ...
3017  1606200192  ...  2ec30b97c371763834305583f4d1709b2f81cfdf399cc5...
3025  1606041387  ...  adfb799b3e5004f804b7b2ecf49e6204729d4bc135bb05...
3039  1606453233  ...  e59659c7fe454bd1a268594ca949fa930d2afbdef3508c...
3041  1601972061  ...  7cb370f95e9f40dc462a9d15aaf77af36eb19c94bc47c1...
3128  1606296202  ...  860422ba909816eacec97a4f09042e53fe1b2b1af39df8...

[314 rows x 11 columns]>

In [ ]:
c = contents["판매수량"].mean()
v = contents["조회수"]
r = contents["판매수량"]
print(c, "", m, "", v, "", r)

22.961649089165867  829.0  0          0
1        130
2        223
3        141
4        136
        ... 
3124     186
3125     105
3126     194
3127       3
3128    3241
Name: 조회수, Length: 3129, dtype: int64  0        0
1        0
2        1
3        0
4        0
        ..
3124     1
3125     0
3126    45
3127     0
3128    26
Name: 판매수량, Length: 3129, dtype: int64


In [ ]:
# weighted rating 식 적용
result = (v / (v+m) * r) + (m / (m+v) * c)
print(result)

0       22.961649
1       19.849017
2       18.306281
3       19.623925
4       19.725603
          ...    
3124    18.937150
3125    20.380307
3126    27.140965
3127    22.878855
3128    25.381132
Length: 3129, dtype: float64


In [ ]:
contents["선호도"] = result
contents

,상품코드,기본분류,분류명,상품유형,상품명,제조사,판매가격,기본배송비,조회수,판매수량,업체ID,선호도
0,157092753,201030,곶감/반건시/감말랭이,NaN,[상주전통곶감] 전통곶감 건시 4kg(65~70과),상주전통곶감,89000,0,0,0,d3f908bbea5d2d7cac866508dff9b2ffdf0088ab55d178...,22.961649
1,1578008979,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시3호 48구,구릿뜰농원,55000,4000,130,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,19.849017
2,1578031964,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시4호 36구,구릿뜰농원,45000,4000,223,1,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,18.306281
3,1578034429,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시 낱개 포장 30구,구릿뜰농원,40000,4000,141,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,19.623925
4,1578034546,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시 낱개 포장1호,구릿뜰농원,30000,4000,136,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,19.725603
...,...,...,...,...,...,...,...,...,...,...,...,...
3124,1604998092,80,김치/장류/참기름,가공식품,바른간장 1.8L,바른된장,24000,3000,186,1,926268d8bc66615558bacbd42caa33ac24a0d0c7d032fd...,18.937150
3125,1606396480,80,김치/장류/참기름,가공식품,두레원무염청국장 1kg,두레원식품,13000,3500,105,0,a279427a10453ba64bc3ff604c06d63769647b4df7a5d5...,20.380307
3126,1606924853,80,김치/장류/참기름,가공식품,두레원무염청국장 200g,두레원식품,3200,3500,194,45,a279427a10453ba64bc3ff604c06d63769647b4df7a5d5...,27.140965
3127,1606196135,b040,침구류,공산품,[풍기인견 아이싱] 사각 끈 메밀베개,풍기인견 아이싱,23000,0,3,0,1e5162a3dd797424426992b764688a7e7759b85a2597d9...,22.878855


# Content Based Filtering
#### 상품 간의 유사도는 상품유형, 제조사, 업체ID


### sklearn의 CountVetorizer를 이용해 분류명, 제조사, 업체ID (띄어쓰기를 구분자로 한 문자열) -> 숫자 벡터화


In [ ]:
contents["업체ID"].head()

0    d3f908bbea5d2d7cac866508dff9b2ffdf0088ab55d178...
1    b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...
2    b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...
3    b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...
4    b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...
Name: 업체ID, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer(ngram_range=(1,3))

c_vector_category = count_vector.fit_transform(contents["업체ID"])
c_vector_category.shape

(3129, 574)

#### sklearn의 cosine similarity를 이용해 코사인 유사도 측정
#### 코사인 유사도를 구함과 동시에, argsort로 유사도가 '가장 높은 인덱스'를 가장 위쪽으로 정렬


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도를 구한 벡터를 미리 저장
category_c_sim = cosine_similarity(c_vector_category, c_vector_category).argsort()[:, ::-1]

In [ ]:
category_c_sim.shape

(3129, 3129)

#### 입력 받은 상품명을 기반으로, 상품과 유사한 다른 상품을 추천하는 함수

In [ ]:
def get_recommend_merchandise_list(df, merc_name, top):

    # 특정 상품과 비슷한 상품을 추천해야 하므로, '특정 상품' 정보 추출
    target_merc_index = df[df["상품명"] == merc_name].index.values

    # 코사인 유사도 중 비슷한 코사인 유사도를 가진 정보 추출
    sim_index = category_c_sim[target_merc_index, :top].reshape(-1)
    # 자기 자신을 제외
    sim_index = sim_index[sim_index != target_merc_index]

    # data frame으로 만들고, '선호도'로 정렬한 뒤 return
    result = df.iloc[sim_index].sort_values('선호도', ascending=False)[:top]
    return result

In [ ]:
# 임의의 상품명
merc_name = "[구릿뜰농원] 건시5호 24구"
top = 5

get_recommend_merchandise_list(contents, merc_name, top)
# 업체ID를 기준으로, 위 상품과 유사한 상품 5개를 추천

,상품코드,기본분류,분류명,상품유형,상품명,제조사,판매가격,기본배송비,조회수,판매수량,업체ID,선호도
33,1585313479,201030,곶감/반건시/감말랭이,농산물,상주구릿뜰곶감 건시5호,구릿뜰농원,28000,4000,6,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,22.796655
1,1578008979,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시3호 48구,구릿뜰농원,55000,4000,130,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,19.849017
4,1578034546,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시 낱개 포장1호,구릿뜰농원,30000,4000,136,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,19.725603
3,1578034429,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시 낱개 포장 30구,구릿뜰농원,40000,4000,141,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,19.623925
2,1578031964,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시4호 36구,구릿뜰농원,45000,4000,223,1,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,18.306281


#### 분류명 외에도 '제조사'와 '업체ID'를 기준으로 콘텐츠 기반 필터링을 이용한 추천 시스템을 구현할 수도 있을듯
#### 상품에 관한 이용자들의 ;선호도;를 임의로 상품의 '조회수'와 '판매수량' 칼럼을 이용해 구했으나, 보다 다양하고, 정확한 맞춤형 추천 기능을 위해서 상품마다 이용자들의 **평점 데이터**를 확보하는 것을 권장함
